Part 1 - Import libraries

In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Libraries imported.


Create Dataframe for Toronto Data - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
def strip_formating(tblRow):
    tblRow = tblRow.text.strip()
    strRow = str(tblRow)
    arrRow = strRow.split('\n')
    return arrRow

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
get_text = requests.get(url).text
soup = BeautifulSoup(get_text,'xml')

NH_tbl = soup.find('table',{'class':'wikitable sortable'})
NH_rw = NH_tbl.find_all('tr')

arr_data = []
for iRow in NH_tbl.find_all('tr'):
    arrRow = strip_formating(iRow)
    arr_data.append(arrRow) 
df = pd.DataFrame(arr_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

Remove rows with Neighborhood = Not assigned, Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 

In [3]:
df1 = df[df.Borough!='Not assigned']
df1 = df1[df1.Borough!='Borough']
df1.reset_index(inplace=True)

If More than one neighborhood can exist in one postal code area combined into one row with the neighborhoods separated with a comma 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [4]:
tempPC = 'MMM'
tempNH = 'NNN'
for x in range(df1.shape[0]):
    if df1.Neighborhood[df1.index==x].values[0] == 'Not assigned':
        df1.at[x,'Neighborhood'] = df1.Borough[df1.index==x].values[0]
        
    if df1.PostalCode[df1.index==x].values[0] == tempPC:  
        df1.at[x,'Neighborhood'] = tempNH + ', ' + df1.Neighborhood[df1.index==x].values[0]
        df1.at[x-1,'Neighborhood'] = 'Delete'
    tempPC =  df1.PostalCode[df1.index==x].values[0]
    tempNH =  df1.Neighborhood[df1.index==x].values[0]
df2 = df1[df1.Neighborhood!='Delete']    

Reindex


In [5]:
df2.reset_index(inplace=True)
del df2['index']
del df2['level_0']
df2.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Use the .shape method to print the number of rows of your dataframe.

In [6]:
df2.shape

(103, 3)

In [ ]:
Part 2 - Add Latitude, Longitude

In [11]:
LatLong = pd.read_csv('http://cocl.us/Geospatial_data')
df3 = pd.merge(df2, LatLong, left_on='PostalCode', right_on='Postal Code')
df3.head(12)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",M5B,43.657162,-79.378937


In [ ]:
Part 3

In [12]:
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


Create Toronto map

In [18]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

ModuleNotFoundError: No module named 'folium'